In [1]:
from dotenv import load_dotenv
load_dotenv()

True

**LLM Initialization**

In [2]:
from crewai import LLM

llm = LLM(
    model = "gemini/gemini-2.0-flash",
    temperature=0.1
)

In [5]:
original_email = """
looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows. ping me if any blockers.
"""

from crewai.tools import BaseTool

class ReplaceJargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description : str = "Replace Jargons with more specific terms"

    def _run(self,email: str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamo project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standard assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }

        suggestions = []
        email_lower = email.lower()
        for jargon,replacemet in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacemet}'")

        return "\n".join(suggestions) if suggestions else "No jargons or internal abbreviation"
    
#Define Tool
jt = ReplaceJargonsTool()

#To check the working of tool
jt.run(original_email)



Using Tool: Replace Jargons


"Consider replacing 'PRX' with 'Project Phoenix (internal AI revamo project)'\nConsider replacing 'TAS' with 'technical architecture stack'\nConsider replacing 'SDS' with 'Smart Data Syncer'\nConsider replacing 'SYNCBOT' with 'internal standard assistant bot'\nConsider replacing 'ping' with 'reach out'"

In [6]:
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role = "email_assistant",
    goal = "Improve emails and make them sound professional and clear",
    backstory = "A highly experienced communication expert skilled in professional email writing",
    tools = [jt],
    verbose = True,
    llm = llm
)



email_task = Task(
    description=f""" Take the following rough email and rewrite it into a professional and polished version.
    '''{original_email}'''
""",
agent = email_assistant,
expected_output="A professional written email with proper formatting and content"
)


crew = Crew(
    agents = [email_assistant],
    tasks = [email_task],
    verbose=True
)

#Run the Crew
result = crew.kickoff()
print(result)

Subject: Updates on Technical Architecture Stack, Project Phoenix, and Smart Data Syncer Integration

Hi Team,

I am including Priya in this email thread.

The updates for the technical architecture stack and Project Phoenix (internal AI revamo project) are available in the deck. The estimated time of arrival for the Smart Data Syncer integration is Friday.

I propose we synchronize our schedules tomorrow, pending the availability of the internal standard assistant bot. Please reach out to me if you encounter any obstacles.

Thank you,

[Your Name]
